# Implementing the Transformer Decoder in Keras

Here again what the decoder does is that it takes the input (target) sequence's "raw" embeddings and, through multiple self-attention + cross-attention layers, gradually _transforms_ them to projections that are more representative and meaningful based on overall context. This is, of course, done in parallel over all tokens all at once. (At inference we generate one token at a time, but we'll get to inference in future chapters).

In [1]:
from numpy import random
from tensorflow import shape
from tensorflow.keras.layers import Dropout, Layer

from xformer.common import AddAndNorm, FeedForward
from xformer.multihead_attention import MultiHeadAttention
from xformer.positional_encoding import CustomEmbeddingWithFixedPosnWts

## 18.1 Recap of the Transformer Decoder

In NLP, typically a sequence-to-sequence model such as translation would have be an encoder+decoder transformer. Once again, the architecture of the decoder has _a lot_ in common with the encoder. Let's focus on its few differences and important details.  

It has _three_ sub-layers instead of two:
1. One multi-head self-attention with queries, keys and values coming from the embedded and positionally-encoded input sequences. This is architecturally identical to the multi-head self-attention layer in the encoder. (Just remember that the inputs come from _target_ sentences).
2. It has an additional multi-head attention sub-layer which is _not_ self-attending. It gets its keys and values coming from the output of the transformer's encoder and it gets is queries from its own multi-head self-attention sub-layer (#1 above). We can call this one multi-head _cross_-attention if you want.
3. As with the encoder, it has a fully-connected feed-forward sub-layer after that.  

As with the encoder, each sub-layer above is followed by an "Add-and-Norm" layer normalization sub-layer. And just as before, regularization is performed by applying a dropout layer to the outputs of each of the above 3 sub-layers right before the normalization step, as well as to the positionally-encoded embeddings right before they are fed into the decoder.

## 18.2 Implementing the Transformer Decoder from Scratch

Let's just jump right into implementing it, starting by defining the `DecoderLayer` and `Decoder` classes, simililarly to how we did things for the encoder and reusing a lot of the code from before.

In [2]:
# Implementing the Decoder Layer
class DecoderLayer(Layer):
    def __init__(self, n_heads, d_model, d_ff, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.multihead_attention1 = MultiHeadAttention(n_heads, d_model)
        self.dropout1 = Dropout(dropout_rate)
        self.add_norm1 = AddAndNorm()
        self.multihead_attention2 = MultiHeadAttention(n_heads, d_model)
        self.dropout2 = Dropout(dropout_rate)
        self.add_norm2 = AddAndNorm()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout3 = Dropout(dropout_rate)
        self.add_norm3 = AddAndNorm()

    def call(self, x, mask, encoder_output, encoder_mask, training):
        # Multi-head self-attention layer
        multihead_output1 = self.multihead_attention1(x, x, x, mask)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        multihead_output1 = self.dropout1(multihead_output1, training=training)

        # Followed by an Add & Norm layer
        addnorm_output1 = self.add_norm1(x, multihead_output1)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Followed by another multi-head (cross-)attention layer
        multihead_output2 = self.multihead_attention2(
            addnorm_output1, encoder_output, encoder_output, encoder_mask
        )

        # Add in another dropout layer
        multihead_output2 = self.dropout2(multihead_output2, training=training)

        # Followed by another Add & Norm layer
        addnorm_output2 = self.add_norm2(addnorm_output1, multihead_output2)

        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(addnorm_output2)
        # Expected output shape = (batch_size, sequence_length, d_model)
        # Add in another dropout layer
        feedforward_output = self.dropout3(
            feedforward_output, training=training
        )
        # Followed by another Add & Norm layer
        return self.add_norm3(addnorm_output2, feedforward_output)

In [3]:
# Implementing the Decoder
class Decoder(Layer):
    def __init__(
        self,
        vocab_size,
        sequence_length,
        n_heads,
        d_model,
        d_ff,
        n_dec_layers,
        dropout_rate,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.wrd_emb_posn_enc = CustomEmbeddingWithFixedPosnWts(
            sequence_length, vocab_size, d_model
        )
        self.dropout = Dropout(dropout_rate)
        self.decoder_layer = [
            DecoderLayer(n_heads, d_model, d_ff, dropout_rate)
            for _ in range(n_dec_layers)
        ]

    def call(
        self,
        target_sequence,
        mask,
        encoder_output,
        encoder_mask,
        training,
    ):
        # Generate the positional encoding
        pos_encoding_output = self.wrd_emb_posn_enc(target_sequence)
        # Expected output shape = (number of sentences, sequence_length, d_model)
        # Add in a dropout layer
        x = self.dropout(pos_encoding_output, training=training)
        # Pass on the positional encoded values to each encoder layer
        for i, layer in enumerate(self.decoder_layer):
            x = layer(x, mask, encoder_output, encoder_mask, training)
        return x

## 18.3 Testing Out the Code

As before, let's test it out with parameter values from AIAYN. We'll use dummy data for the target sequences _and_ for our encoder output. Also we won't be using masks yet.

In [4]:
h = 8  # Number of self-attention heads
d_ff = 2048  # Dimensionality of the inner fully-connected layer
d_model = 512  # Dimensionality of the model
n = 6  # Number of layers in the encoder stack

batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in dropout layers

dec_vocab_size = 20  # Vocabulary size for the decoder
input_seq_length = 5  # Maximum length of the input sequence
input_seq = random.random((batch_size, input_seq_length))
enc_output = random.random((batch_size, input_seq_length, d_model))

decoder = Decoder(
    dec_vocab_size, input_seq_length, h, d_model, d_ff, n, dropout_rate
)

print(decoder(input_seq, None, enc_output, None, True))

tf.Tensor(
[[[ 0.14689298  1.4889189   1.4373505  ...  0.2518994  -1.1189088
   -1.5063888 ]
  [ 0.36743933  0.85430753  0.62036705 ...  0.05930083 -0.34579793
   -1.7476729 ]
  [-0.42957708  1.2475446   1.4967277  ...  0.10902691 -1.3981266
   -0.5514256 ]
  [-0.73810756  1.0564915   0.44105342 ...  0.24699281 -1.1235929
   -1.281653  ]
  [-0.7004271   0.7580604   0.511743   ...  0.04367047 -0.69809765
   -1.1786984 ]]

 [[-0.33187953  1.6393027   0.7590318  ...  0.10175119 -0.5207812
   -0.96342885]
  [ 0.15893792  1.5600148   1.3638816  ... -0.28914726 -0.15736698
   -1.3809341 ]
  [-0.1530173   0.9120058   0.9395371  ... -0.4249657  -0.03356398
   -1.9221315 ]
  [ 0.10429339  1.4003291   1.7235233  ... -0.1958521   0.2185743
   -1.5828359 ]
  [ 0.57673734  2.2125227  -0.12343413 ...  0.39349705 -0.30001473
   -1.1428877 ]]

 [[ 0.39791936  0.07334416  0.9556452  ... -0.02623349 -0.5667957
   -2.1628373 ]
  [ 0.71776915  0.7817034   0.40280566 ... -0.6789003   0.02750724
   -1.67765